# 4-Oscar Prediction with AutoML
After out dataframe has been assemlbed (see scraping and table_assembling) notebooks we have the data we need to make predictions on the Best Picture winner. [AutoML](http://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html) represents a quick, but powerful route though the Machine Learning process. H2O's AutoML runs many models through the dataset and using cross-validation, picks the best one. For my purposes I use it to confirm/compare to the Preferential Balloting Random Forest model I created.
If you are gunning to win your office's Oscar pool, scroll down to see the results.

In [1]:
import pandas as pd
import numpy as np
import h2o

# Machine Learning - Using h2o Auto ML

In [2]:
full_table = pd.read_csv('../data/processed_results/extended_df.csv')
full_table=full_table.drop('Unnamed: 0', axis=1)
full_table

,year,film,wiki,winner,rating,numVotes,worldwide_box_office,Action,Adventure,Animation,...,nom_pga,winner_pga,nom_bafta,winner_bafta,nom_dga,winner_dga,nom_sag,winner_sag,nom_cannes,winner_cannes
0,1927,Wings,/wiki/Wings_(1927_film),True,7.3,13576.0,$746,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1927,7th Heaven,/wiki/7th_Heaven_(1927_film),False,5.2,26223.0,"$79,808",0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1927,The Racket,/wiki/The_Racket_(1928_film),False,6.7,3149.0,"$21,733,230",0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1928,The Broadway Melody,/wiki/The_Broadway_Melody,True,5.6,7605.0,"$223,723",0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1928,Alibi,/wiki/Alibi_(1929_film),False,7.4,391.0,"$42,915",0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,2022,The Fabelmans,/wiki/The_Fabelmans,False,7.6,85709.0,"$45,164,110",0,0,0,...,1,0,0,0,1,0,1,0,0,0
547,2022,Tár,/wiki/T%C3%A1r,False,7.5,69684.0,"$27,541,681",0,0,0,...,1,0,1,0,1,0,0,0,0,0
548,2022,Top Gun: Maverick,/wiki/Top_Gun:_Maverick,False,8.3,577408.0,"$1,493,491,858",0,0,0,...,1,0,0,0,1,0,0,0,0,0
549,2022,Triangle of Sadness,/wiki/Triangle_of_Sadness,False,7.3,128812.0,"$25,615,870",0,0,0,...,0,0,0,0,0,0,0,0,1,1


In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,1 day 17 hours 8 mins
H2O_cluster_timezone:,Europe/Belgrade
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.4
H2O_cluster_version_age:,20 days
H2O_cluster_name:,H2O_from_python_aczaplak_wkb9zy
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.524 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


First Year of Existance. This data will be used below
- golden_globes 1943
- pga 1989
- bafta 1960
- dga 1948
- sag 1995
- cannes 1970

In [4]:
# I pick a min_year where the awards shows will be relevant
min_year = 1995

# H2O's Auto ML

In [5]:
# Auto ML uses Cross Validation, so we do not specifiy a validation set
train = full_table.loc[((full_table['year'] < 2022) & (full_table['year'] >= min_year))]

print('training set contains:', train.shape[0], 'movies')

training set contains: 184 movies


In [6]:
#train = train.drop(['index', '[]'], axis=1)
train.columns

Index(['year', 'film', 'wiki', 'winner', 'rating', 'numVotes',
       'worldwide_box_office', 'Action', 'Adventure', 'Animation', 'Biography',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy',
       'Film-Noir', 'Game-Show', 'History', 'Horror', 'Music', 'Musical',
       'Mystery', 'News', 'Reality-TV', 'Romance', 'Sci-Fi', 'Short', 'Sport',
       'Talk-Show', 'Thriller', 'War', 'Western', 'action', 'adventure',
       'animation', 'biography', 'comedy', 'crime', 'documentary', 'drama',
       'family', 'fantasy', 'film-noir', 'history', 'horror', 'music',
       'musical', 'mystery', 'romance', 'sci-fi', 'sport', 'thriller', 'war',
       'western', 'nominations', 'Oscar_win', 'nom_gg_drama',
       'winner_gg_drama', 'nom_gg_comedy', 'winner_gg_comedy', 'nom_pga',
       'winner_pga', 'nom_bafta', 'winner_bafta', 'nom_dga', 'winner_dga',
       'nom_sag', 'winner_sag', 'nom_cannes', 'winner_cannes'],
      dtype='object')

In [6]:
print(type(train))

<class 'pandas.core.frame.DataFrame'>


In [7]:
from h2o.automl import H2OAutoML, get_leaderboard

# Import a sample binary outcome train/test set into H2O
train1 = h2o.H2OFrame(train)

# Identify predictors and response
predictors = ['year', 'film', 'wiki', 'rating', 'numVotes',
       'worldwide_box_office', 'Action', 'Adventure', 'Animation', 'Biography',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy',
       'Film-Noir', 'Game-Show', 'History', 'Horror', 'Music', 'Musical',
       'Mystery', 'News', 'Reality-TV', 'Romance', 'Sci-Fi', 'Short', 'Sport',
       'Talk-Show', 'Thriller', 'War', 'Western', 'action', 'adventure',
       'animation', 'biography', 'comedy', 'crime', 'documentary', 'drama',
       'family', 'fantasy', 'film-noir', 'history', 'horror', 'music',
       'musical', 'mystery', 'romance', 'sci-fi', 'sport', 'thriller', 'war',
       'western', 'nominations', 'nom_gg_drama',
       'winner_gg_drama', 'nom_gg_comedy', 'winner_gg_comedy', 'nom_pga',
       'winner_pga', 'nom_bafta', 'winner_bafta', 'nom_dga', 'winner_dga',
       'nom_sag', 'winner_sag', 'nom_cannes', 'winner_cannes']

x = predictors
y = 'Oscar_win'

# For binary classification, response should be a factor
train1[y] = train1[y].asfactor()

# Run AutoML for 100 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=100, seed=1
                , keep_cross_validation_predictions= True
               , exclude_algos = ['StackedEnsemble'])

aml.train(x=x, y=y, training_frame=train1)

# AutoML Leaderboard
lb = aml.leaderboard

# Optionally edd extra model information to the leaderboard
lb = get_leaderboard(aml, extra_columns='ALL')

# Print all rows (instead of default 10 rows)
lb.head(rows=lb.nrows)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |█
09:19:06.887: AutoML: XGBoost is not available; skipping it.
09:19:06.903: _train param, Dropping bad and constant columns: [Film-Noir, Sport, Horror, winner_cannes, worldwide_box_office, wiki, nom_cannes, film, film-noir, documentary]
09:19:07.410: _train param, Dropping bad and constant columns: [Film-Noir, Sport, Horror, winner_cannes, worldwide_box_office, wiki, nom_cannes, film, film-noir, documentary]
09:19:07.410: _min_rows param, The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 184.0.
09:19:07.417: _train param, Dropping bad and constant columns: [Film-Noir, Sport, Horror, winner_cannes, worldwide_box_office, wiki, nom_cannes, film, film-noir, documentary]
09:19:08.228: _train param, Dropping bad and constant columns: [Film-Noir, Sport, Horror, winner_cannes, worldwide_box_office, wiki, nom_cannes, f

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,training_time_ms,predict_time_per_row_ms,algo
DeepLearning_grid_2_AutoML_6_20230519_91906_model_4,0.700873,0.674915,0.28838,0.334277,0.4014,0.161122,5273,0.182118,DeepLearning
DeepLearning_grid_1_AutoML_6_20230519_91906_model_3,0.674923,2.39876,0.2879,0.335221,0.440234,0.193806,3483,0.100092,DeepLearning
DeepLearning_grid_1_AutoML_6_20230519_91906_model_7,0.673508,0.687366,0.254215,0.327908,0.398665,0.158934,8082,0.062039,DeepLearning
DeepLearning_grid_3_AutoML_6_20230519_91906_model_19,0.6728,0.738511,0.201342,0.297594,0.394644,0.155744,5373,0.091604,DeepLearning
DeepLearning_grid_1_AutoML_6_20230519_91906_model_15,0.66419,1.86933,0.286557,0.344067,0.443692,0.196863,2558,0.055437,DeepLearning
DeepLearning_grid_2_AutoML_6_20230519_91906_model_14,0.661005,0.906602,0.221235,0.299009,0.40559,0.164503,4761,0.073049,DeepLearning
DeepLearning_grid_1_AutoML_6_20230519_91906_model_8,0.643312,0.734009,0.217925,0.358575,0.397667,0.158139,10236,0.064391,DeepLearning
DeepLearning_grid_1_AutoML_6_20230519_91906_model_23,0.63482,1.02315,0.205258,0.34218,0.514426,0.264634,3730,0.076604,DeepLearning
DeepLearning_grid_1_AutoML_6_20230519_91906_model_14,0.633876,0.887214,0.213518,0.36058,0.394216,0.155406,2718,0.060776,DeepLearning
DeepLearning_grid_3_AutoML_6_20230519_91906_model_20,0.627506,0.489866,0.19211,0.336046,0.384056,0.147499,2643,0.062382,DeepLearning


In [19]:
lb.as_data_frame().to_csv('./model/leaderboard.csv')

In [8]:
top_model = aml.leader
top_model

Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_grid_2_AutoML_6_20230519_91906_model_4


Status of Neuron Layers: predicting Oscar_win, 2-class classification, bernoulli distribution, CrossEntropy loss, 16 402 weights/biases, 205,0 KB, 55 200 training samples, mini-batch size 1
    layer    units    type              dropout    l1    l2    mean_rate              rate_rms              momentum    mean_weight            weight_rms           mean_bias                bias_rms
--  -------  -------  ----------------  ---------  ----  ----  ---------------------  --------------------  ----------  ---------------------  -------------------  -----------------------  --------------------
    1        60       Input             10.0
    2        100      RectifierDropout  0.0        0.0   0.0   0.012195518964182459   0.02024739980697632   0.0         0.0023191050170753443  0.11466699838638306  0.49353053754016796      0.05457335710525513
    3        100      RectifierDropout  0.0        0.0   0.0   0.013878522963631257   0.049991458654403687  0.0         -0.001626654772080201  0.10583999752998352  0.9973973050687307       0.015333399176597595
    4        2        Softmax                      0.0   0.0   0.0027545143297174946  0.007758587598800659  0.0         0.027643022865522653   0.5707516670227051   -0.00017305792290297876  0.007940754294395447

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.004111662465069766
RMSE: 0.06412224625720597
LogLoss: 0.013626588173579179
Mean Per-Class Error: 0.0
AUC: 1.0
AUCPR: 1.0
Gini: 1.0

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.2125439532782935
       0    1    Error    Rate
-----  ---  ---  -------  -----------
0      157  0    0        (0.0/157.0)
1      0    27   0        (0.0/27.0)
Total  157  27   0        (0.0/184.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.212544     1         26
max f2                       0.212544     1         26
max f0point5                 0.212544     1         26
max accuracy                 0.212544     1         26
max precision                1            1         0
max recall                   0.212544     1         26
max specificity              1            1         0
max absolute_mcc             0.212544     1         26
max min_per_class_accuracy   0.212544     1         26
max mean_per_class_accuracy  0.212544     1         26
max tns                      1            157       0
max fns                      1            26        0
max fps                      7.8358e-15   157       183
max tps                      0.212544     27        26
max tnr                      1            1         0
max fnr                      1            0.962963  0
max fpr                      7.8358e-15   1         183
max tpr                      0.212544     1         26

Gains/Lift Table: Avg response rate: 14,67 %, avg score: 14,05 %
group    cumulative_data_fraction    lower_threshold    lift     cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain     cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  -------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  -------  -----------------  --------------------
1        0.0108696                   0.999999           6.81481  6.81481            1                1            1                           1                   0.0740741       0.0740741                  581.481  581.481            0.0740741
2        0.0217391                   0.999997           6.81481  6.81481            1                0.999999     1                           0.999999    

In [9]:
h2o.save_model(top_model, './additional_data/model.zip' )

'C:\\Users\\aczaplak.MPD1\\repos\\oscar_predictions\\additional_data\\additional_data\\model.zip\\DeepLearning_grid_2_AutoML_6_20230519_91906_model_4'

In [10]:
model = h2o.load_model('./additional_data/model.zip/DeepLearning_grid_2_AutoML_6_20230519_91906_model_4')

In [11]:
model

Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_grid_2_AutoML_6_20230519_91906_model_4


Status of Neuron Layers: predicting Oscar_win, 2-class classification, bernoulli distribution, CrossEntropy loss, 16 402 weights/biases, 205,0 KB, 55 200 training samples, mini-batch size 1
    layer    units    type              dropout    l1    l2    mean_rate              rate_rms              momentum    mean_weight            weight_rms           mean_bias                bias_rms
--  -------  -------  ----------------  ---------  ----  ----  ---------------------  --------------------  ----------  ---------------------  -------------------  -----------------------  --------------------
    1        60       Input             10.0
    2        100      RectifierDropout  0.0        0.0   0.0   0.012195518964182459   0.02024739980697632   0.0         0.0023191050170753443  0.11466699838638306  0.49353053754016796      0.05457335710525513
    3        100      RectifierDropout  0.0        0.0   0.0   0.013878522963631257   0.049991458654403687  0.0         -0.001626654772080201  0.10583999752998352  0.9973973050687307       0.015333399176597595
    4        2        Softmax                      0.0   0.0   0.0027545143297174946  0.007758587598800659  0.0         0.027643022865522653   0.5707516670227051   -0.00017305792290297876  0.007940754294395447

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.004111662465069766
RMSE: 0.06412224625720597
LogLoss: 0.013626588173579179
Mean Per-Class Error: 0.0
AUC: 1.0
AUCPR: 1.0
Gini: 1.0

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.2125439532782935
       0    1    Error    Rate
-----  ---  ---  -------  -----------
0      157  0    0        (0.0/157.0)
1      0    27   0        (0.0/27.0)
Total  157  27   0        (0.0/184.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.212544     1         26
max f2                       0.212544     1         26
max f0point5                 0.212544     1         26
max accuracy                 0.212544     1         26
max precision                1            1         0
max recall                   0.212544     1         26
max specificity              1            1         0
max absolute_mcc             0.212544     1         26
max min_per_class_accuracy   0.212544     1         26
max mean_per_class_accuracy  0.212544     1         26
max tns                      1            157       0
max fns                      1            26        0
max fps                      7.8358e-15   157       183
max tps                      0.212544     27        26
max tnr                      1            1         0
max fnr                      1            0.962963  0
max fpr                      7.8358e-15   1         183
max tpr                      0.212544     1         26

Gains/Lift Table: Avg response rate: 14,67 %, avg score: 14,05 %
group    cumulative_data_fraction    lower_threshold    lift     cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain     cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  -------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  -------  -----------------  --------------------
1        0.0108696                   0.999999           6.81481  6.81481            1                1            1                           1                   0.0740741       0.0740741                  581.481  581.481            0.0740741
2        0.0217391                   0.999997           6.81481  6.81481            1                0.999999     1                           0.999999    

## Predict the winner

In [13]:
# Predict on 2019's films
test = full_table.loc[(full_table['year'] == 2021)]

# Import a binary outcome train/test set into H2O
test = h2o.H2OFrame(test)

# For binary classification, response should be a factor
test[y] = test[y].asfactor()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [14]:
preds = top_model.predict(test)

preds

deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%


predict,p0,p1
0,0.999976,2.37118e-05
1,0.00165605,0.998344
0,0.995174,0.00482619
0,1,1.74511e-07
0,0.995249,0.00475117
0,0.999972,2.78123e-05
0,0.999979,2.0885e-05
0,1,9.07248e-09
0,0.999955,4.5392e-05
0,0.999999,1.10776e-06


In [15]:
test['pred'] = preds['predict']
test['probA'] = preds['p1']
test_pd = test.as_data_frame(use_pandas=True)

In [16]:
final_rankings = test_pd[['film','probA']].sort_values('probA', ascending = False)
final_rankings['%_confidence'] = final_rankings['probA']/final_rankings['probA'].sum() * 100
final_rankings

,film,probA,%_confidence
1,CODA,9.983440e-01,9.903809e+01
2,Belfast,4.826187e-03,4.787692e-01
4,Drive My Car,4.751175e-03,4.713278e-01
8,Nightmare Alley,4.539204e-05,4.502998e-03
5,Dune,2.781235e-05,2.759051e-03
0,West Side Story,2.371181e-05,2.352268e-03
6,King Richard,2.088495e-05,2.071837e-03
9,The Power of the Dog,1.107759e-06,1.098923e-04
3,Don't Look Up,1.745112e-07,1.731192e-05
7,Licorice Pizza,9.072485e-09,9.000120e-07


# And the Oscar goes to...

In [13]:
bp_winner = np.array(final_rankings.reset_index())[0][1].split('(')[0].strip()
print(f'And the Oscar goes to...\n🎉🏆{bp_winner}🏆🎉')

And the Oscar goes to...
🎉🏆1917🏆🎉
